# LangGraph Tutorial: Setup & Validation

## Objective
Set up your development environment for LangGraph and validate connectivity to the LLM provider.

## What You'll Learn
1. Required dependencies for LangGraph development
2. Environment variable configuration
3. LLM initialization with Google's Gemini
4. Connectivity testing and validation
5. Optional: LangSmith observability setup

## Prerequisites
- Python 3.10+ installed
- A Google AI Studio API key (free tier available)
- Basic familiarity with virtual environments

---

## Section 1: Installation

### Required Packages

Run this command in your terminal to install all dependencies:

```bash
pip install langchain langchain-google-genai langchain-core langgraph python-dotenv
```

### Reference Point: Package Purposes

| Package | Purpose |
|---------|--------|
| `langchain` | Core framework for building LLM applications |
| `langchain-google-genai` | Google Gemini integration |
| `langchain-core` | Core abstractions (tools, messages, etc.) |
| `langgraph` | Graph-based agent workflows |
| `python-dotenv` | Load environment variables from `.env` files |

---

## Section 2: Environment Configuration

### Create a `.env` File

Create a `.env` file in your project root with the following content:

```env
# ===========================================
# REQUIRED: Google AI Configuration
# ===========================================
# Get your API key from: https://aistudio.google.com/apikey
GOOGLE_API_KEY=your-google-api-key-here

# ===========================================
# OPTIONAL: LangSmith Observability
# ===========================================
# Get your API key from: https://smith.langchain.com
LANGSMITH_API_KEY=your-langsmith-key
LANGSMITH_PROJECT=langgraph-tutorial
LANGCHAIN_TRACING_V2=true
```

### Reference Point: Getting Your API Key

1. Go to [Google AI Studio](https://aistudio.google.com/apikey)
2. Sign in with your Google account
3. Click "Create API Key"
4. Copy the key and paste it in your `.env` file

> **Security Note:** Never commit your `.env` file to version control. Add `.env` to your `.gitignore` file.

---

## Section 3: Import Dependencies

Let's import all the packages we'll need for this tutorial series.

In [1]:
# LLM Provider
from langchain_google_genai import ChatGoogleGenerativeAI

# Environment management
import os
from dotenv import load_dotenv

print("✅ All imports successful")

✅ All imports successful


### Reference Point: Import Categories

As we progress through the tutorial, we'll add more imports:

| Category | Imports | Used In |
|----------|---------|--------|
| **LLM** | `ChatGoogleGenerativeAI` | This notebook |
| **Tools** | `tool` from `langchain_core.tools` | Notebook 02, 03, 04 |
| **Graph** | `StateGraph`, `MessagesState` from `langgraph` | Future notebooks |
| **Messages** | `HumanMessage`, `AIMessage` from `langchain_core` | Future notebooks |

---

## Section 4: Load Environment Variables

Load the variables from your `.env` file into the runtime environment.

In [2]:
# Load environment variables from .env file
# Adjust the path based on your .env file location
load_dotenv("../../.env")  # Change this path if needed

print("✅ Environment file loaded")

✅ Environment file loaded


In [3]:
# Validate required environment variables
def check_env_variable(var_name, required=True):
    """Check if an environment variable is set."""
    value = os.getenv(var_name)
    if value:
        # Mask the value for security (show first 4 and last 4 chars)
        if len(value) > 12:
            masked = f"{value[:4]}...{value[-4:]}"
        else:
            masked = "****"
        print(f"   ✅ {var_name}: {masked}")
        return True
    else:
        if required:
            print(f"   ❌ {var_name}: NOT SET (Required!)")
        else:
            print(f"   ⚠️  {var_name}: Not set (Optional)")
        return False

print("Environment Variables Status:")
print("=" * 50)

# Check required variables
print("\nRequired:")
api_key_ok = check_env_variable("GOOGLE_API_KEY", required=True)

# Check optional variables
print("\nOptional (LangSmith):")
check_env_variable("LANGSMITH_API_KEY", required=False)
check_env_variable("LANGSMITH_PROJECT", required=False)

tracing = os.getenv("LANGCHAIN_TRACING_V2")
print(f"   {'✅' if tracing else '⚠️ '} LANGCHAIN_TRACING_V2: {'Enabled' if tracing else 'Disabled'}")

print("\n" + "=" * 50)
if api_key_ok:
    print("✅ Required configuration complete!")
else:
    print("❌ Missing required configuration. Please check your .env file.")

Environment Variables Status:

Required:
   ✅ GOOGLE_API_KEY: AIza...UgrE

Optional (LangSmith):
   ✅ LANGSMITH_API_KEY: lsv2...dee0
   ✅ LANGSMITH_PROJECT: car-...-rag
   ⚠️  LANGCHAIN_TRACING_V2: Disabled

✅ Required configuration complete!


### Reference Point: Troubleshooting Environment Issues

| Problem | Solution |
|---------|----------|
| `GOOGLE_API_KEY: NOT SET` | Check `.env` file path in `load_dotenv()` call |
| File not found | Verify `.env` file exists and path is correct |
| Key shows but LLM fails | Verify API key is valid at [AI Studio](https://aistudio.google.com) |
| Import errors | Run `pip install` command from Section 1 |

---

## Section 5: Initialize the LLM

Now let's create our LLM instance using Google's Gemini model.

In [4]:
# Create the LLM instance
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized successfully")
print(f"   Model: gemini-2.0-flash")
print(f"   Temperature: 0.3")
print(f"   Max Tokens: 1024")

✅ LLM initialized successfully
   Model: gemini-2.0-flash
   Temperature: 0.3
   Max Tokens: 1024


### Reference Point: LLM Configuration Options

| Parameter | Value | Purpose |
|-----------|-------|--------|
| `model` | `gemini-2.0-flash` | Fast, capable model suitable for tool use |
| `temperature` | `0.3` | Lower = more deterministic responses (good for tools) |
| `max_tokens` | `1024` | Maximum response length |

### Available Gemini Models

| Model | Best For |
|-------|----------|
| `gemini-2.0-flash` | Fast responses, good for most tasks |
| `gemini-2.5-pro` | Complex reasoning, longer context |
| `gemini-2.5-flash` | Balance of speed and capability |

> **Tip:** Start with `gemini-2.0-flash` for development (faster, cheaper), then upgrade to `gemini-2.5-pro` for production if needed.

---

## Section 6: Test LLM Connectivity

Let's verify the LLM is working correctly by sending a simple test message.

In [5]:
print("=" * 50)
print("LLM Connectivity Test")
print("=" * 50)

try:
    # Send a simple test message
    test_response = llm.invoke("Say 'Hello from LangGraph!' if you can read this.")
    
    print(f"\n🧪 Test Prompt: 'Say Hello from LangGraph!'")
    print(f"\n🤖 Response: {test_response.content}")
    print("\n" + "=" * 50)
    print("✅ LLM connectivity verified!")
    
except Exception as e:
    print(f"\n❌ Error: {type(e).__name__}")
    print(f"   Message: {e}")
    print("\n" + "=" * 50)
    print("❌ LLM connectivity failed. See troubleshooting below.")

LLM Connectivity Test

🧪 Test Prompt: 'Say Hello from LangGraph!'

🤖 Response: Hello from LangGraph!

✅ LLM connectivity verified!


### Reference Point: Common Connectivity Errors

| Error | Cause | Solution |
|-------|-------|----------|
| `AuthenticationError` | Invalid API key | Regenerate key at AI Studio |
| `RateLimitError` | Too many requests | Wait and retry, or upgrade plan |
| `ConnectionError` | Network issues | Check internet connection |
| `InvalidRequestError` | Malformed request | Check model name spelling |

---

## Section 7: Understanding the Response Object

Let's examine what the LLM returns. This will be important when we integrate tools later.

In [6]:
# Make another call to examine the response structure
response = llm.invoke("What is 2 + 2?")

print("Response Object Analysis")
print("=" * 50)
print(f"\nType: {type(response)}")
print(f"\nContent: {response.content}")
print(f"\nResponse Metadata:")
for key, value in response.response_metadata.items():
    print(f"   {key}: {value}")

Response Object Analysis

Type: <class 'langchain_core.messages.ai.AIMessage'>

Content: 2 + 2 = 4

Response Metadata:
   finish_reason: STOP
   model_name: gemini-2.0-flash
   safety_ratings: []
   model_provider: google_genai


In [7]:
response

AIMessage(content='2 + 2 = 4', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c1c0a-b7a7-7ad2-9273-ee8c65de386c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

### Reference Point: AIMessage Structure

The LLM returns an `AIMessage` object with these key attributes:

| Attribute | Description |
|-----------|-------------|
| `.content` | The text response from the LLM |
| `.response_metadata` | Model info, token usage, finish reason |
| `.tool_calls` | List of tool calls (empty if no tools used) |
| `.id` | Unique message identifier |

> **Important for Tools:** When we add tools later, the LLM will populate `.tool_calls` instead of (or in addition to) `.content`. This is how the LLM requests tool execution.

---

## Section 8: Optional - LangSmith Tracing

LangSmith provides observability for your LLM applications. If you've configured it, let's verify it's working.

In [8]:
# Check LangSmith configuration
langsmith_enabled = os.getenv("LANGCHAIN_TRACING_V2") == "true"
langsmith_project = os.getenv("LANGSMITH_PROJECT", "default")

print("LangSmith Status")
print("=" * 50)

if langsmith_enabled:
    print(f"\n✅ LangSmith tracing is ENABLED")
    print(f"   Project: {langsmith_project}")
    print(f"\n   View traces at: https://smith.langchain.com")
    print(f"   Look for project: '{langsmith_project}'")
else:
    print(f"\n⚠️  LangSmith tracing is DISABLED")
    print(f"   To enable, add to your .env file:")
    print(f"   LANGCHAIN_TRACING_V2=true")
    print(f"   LANGSMITH_API_KEY=your-key")

LangSmith Status

⚠️  LangSmith tracing is DISABLED
   To enable, add to your .env file:
   LANGCHAIN_TRACING_V2=true
   LANGSMITH_API_KEY=your-key


### Reference Point: Why Use LangSmith?

LangSmith helps you:

| Feature | Benefit |
|---------|--------|
| **Trace Visualization** | See the full execution flow of your agent |
| **Token Usage** | Track costs across all LLM calls |
| **Debugging** | Identify where errors occur in complex chains |
| **Latency Analysis** | Find performance bottlenecks |

> **Recommendation:** Enable LangSmith during development. It's invaluable for debugging tool-calling agents.

---

## Summary

In this notebook, you completed:

| Step | Status |
|------|--------|
| Installed dependencies | ✅ |
| Configured environment variables | ✅ |
| Imported required packages | ✅ |
| Initialized the LLM | ✅ |
| Verified connectivity | ✅ |
| Understood response structure | ✅ |

## Tutorial Series Overview

You're now ready to proceed through the series:

| Notebook | Topic | Description |
|----------|-------|-------------|
| **01** (this) | Setup & Validation | Environment configuration ✅ |
| **02** | Getting Started with Tools | `@tool` decorator, `.invoke()`, schemas |
| **03** | Currency Converter | Multi-parameter tool with validation |
| **04** | EMI Calculator | Complex calculations, mixed types |

## Next Steps

Proceed to **Notebook 02: Getting Started with LangGraph Tools** to learn how to create tools that extend LLM capabilities.

---

## Quick Reference: Reusable Code

Copy this setup code to future notebooks:

In [ ]:
# ========================================
# STANDARD SETUP - Copy to new notebooks
# ========================================

# Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from dotenv import load_dotenv
import os

# Load environment
load_dotenv("../../.env")  # Adjust path as needed

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ Setup complete")